In [4]:
import random
import itertools
import operator
import numpy as np 
import scipy as sp
import scipy.linalg as spla
np.set_printoptions(precision=4, linewidth=np.nan)

In [10]:
def nf2DualLP(filename):
    
    # assumes that first row is source and last is sink like in the question
    # edges will be numbered as if they are being read row-by-row left to right
    # vertices will be numbered by row
    
    nf = np.loadtxt(filename)
    for i in range(nf.shape[0]):
        nf[i, i] = 0
    numedges = np.count_nonzero(nf)
    numvertices = nf.shape[0] - 2     # non terminal vertices
    numslacks = numedges
    slack_counter = 0
    edge_counter = 0
    dual_constraints = np.zeros((numedges, numedges + numvertices + numslacks + 1))
    obj = np.zeros(2 * numedges + numvertices)
    
    for i in range(numvertices + 2):
        for j in range(numvertices + 2):
            if nf[i, j] != 0:

                obj[edge_counter] = nf[i, j]
                
                if i == 0:
                    dual_constraints[edge_counter, edge_counter] = 1
                    dual_constraints[edge_counter, numedges + j - 1] = 1
                    dual_constraints[edge_counter, numedges + numvertices + slack_counter] = -1
                    dual_constraints[edge_counter, -1] = 1
                    edge_counter+=1
                    slack_counter+=1
                
                elif j == numvertices + 1:
                    dual_constraints[edge_counter, edge_counter] = 1
                    dual_constraints[edge_counter, numedges + i - 1] = -1
                    dual_constraints[edge_counter, numedges + numvertices + slack_counter] = -1
                    dual_constraints[edge_counter, -1] = 0
                    edge_counter+=1
                    slack_counter+=1
                    
                else:
                    dual_constraints[edge_counter, edge_counter] = 1
                    dual_constraints[edge_counter, numedges + i - 1] = -1
                    dual_constraints[edge_counter, numedges + j - 1] = 1
                    dual_constraints[edge_counter, numedges + numvertices + slack_counter] = -1
                    edge_counter+=1
                    slack_counter+=1
    
    sign_constraints = np.block([
        [np.eye(numedges), np.zeros((numedges, numvertices + numslacks + 1))],
        [np.zeros((numslacks, numedges + numvertices)), np.eye(numslacks), np.ones(numedges).reshape(1, numedges).T]
    ])
    
    LPMatrix = np.vstack((dual_constraints, sign_constraints))
    
    
    return LPMatrix, obj
                    
def nf2PrimalLP(filename):
    nf = np.loadtxt(filename)
    for i in range(nf.shape[0]):
        nf[i, i] = 0
    numedges = np.count_nonzero(nf)
    numvertices = nf.shape[0] - 2
    numslacks = numedges
    slack_counter = 0
    edge_counter = 0
    
    primal_constraints = np.zeros((numedges + numvertices + 2, numedges + numslacks + 1))
    
    obj = np.zeros(numedges + numslacks)
    
    for i in range(numvertices + 2):
        for j in range(numvertices + 2):
            if nf[i, j] != 0:
                if i == 0:
                    obj[edge_counter] = -1
                    primal_constraints[edge_counter, edge_counter] = 1
                    primal_constraints[edge_counter, numedges + slack_counter] = 1
                    primal_constraints[edge_counter, -1] = nf[i, j]
                    primal_constraints[numedges + j, edge_counter] = 1 
                    edge_counter+=1
                    slack_counter+=1
                elif j == numvertices + 1:
                    primal_constraints[edge_counter, edge_counter] = 1
                    primal_constraints[edge_counter, numedges + slack_counter] = 1
                    primal_constraints[edge_counter, -1] = nf[i, j]
                    primal_constraints[numedges + i, edge_counter] = -1
                    edge_counter+=1
                    slack_counter+=1
                else:
                    primal_constraints[edge_counter, edge_counter] = 1
                    primal_constraints[edge_counter, numedges + slack_counter] = 1
                    primal_constraints[edge_counter, -1] = nf[i, j]
                    primal_constraints[numedges + i, edge_counter] = -1
                    primal_constraints[numedges + j, edge_counter] = 1
                    edge_counter+=1
                    slack_counter+=1
    
    sign_constraints = np.hstack((np.eye(2*numedges), np.zeros(2*numedges).reshape(1, 2*numedges).T))
    
    LPMatrix = np.vstack((primal_constraints, sign_constraints))
    
    return LPMatrix, obj


In [19]:
class LPSolution(object):
    
    def __init__(self, num_vars=0, var_vals=list(), obj=0):
        
        self.num_vars = num_vars
        self.obj = obj
        self.var_vals = var_vals
        
    def __str__(self):
        
        sol = ""
        sol += "\tSolution to the LP is as follows:\n\n"
        sol += "\t\toptim\t:=\t" + str(self.obj) + "\n\n"
        for i in range(self.num_vars):
            if i in self.var_vals:
                sol += "\t\tx_" + str(i + 1) + "*\t:=\t" + str(self.var_vals[i]) + "\n"
            else:
                sol += "\t\tx_" + str(i + 1) + "*\t:=\t" + str(0.0) + "\n"
        return sol

In [20]:
class Simplex(object):
    # num_eq_constraints : no. of equality constraints
    def __init__(self, num_eq_constraints, num_vars, objective, constraints=None, max_iter=100):

        self.num_eq_constraints = num_eq_constraints
        self.num_vars = num_vars
        self.c = objective
        if constraints is not None:
            self.constraints = constraints
            self.A = self.constraints[:self.num_eq_constraints, :-1]
            self.b = self.constraints[:self.num_eq_constraints, -1]
            

        else:
            self.A = None
            self.b = None
            self.basic_columns = None
        self.solution = None
        self.tableau = None
        self.max_iter = max_iter
        
    
    def set_constraints(self, const):
        self.constraints = const
    
    def get_num_constraints(self):
        return self.num_constraints
    
    def get_num_vars(self):
        return self.num_vars
    
    def get_constraints(self):
        return self.constraints
    
    
    def fetch_constraints(self, filename):
        self.constraints = np.loadtxt(filename)
        self.b = self.constraints[:self.num_eq_constraints, -1]
        self.constraints  = self.constraints[:, :-1]
        self.A = self.constraints[:self.num_eq_constraints, :]

    
    def get_sol(self):
        return self.sol
    

    
    def get_first_basic_columns(self):
        basic_columns = random.choice(self.basic_columns)
        return basic_columns
    
    def get_first_B(self, basic_cols):
        return self.A[:, basic_cols]
    
    def run_phase1(self):
        
        c = np.hstack([np.zeros(self.A.shape[1]), np.ones(self.A.shape[0])])
        ph1_tableau = None
        A = np.hstack([self.A.copy(), np.eye(self.A.shape[0])])
        basic_columns = (self.A.shape[1]) + np.arange(self.A.shape[0])
        B = A[:, basic_columns]
        b = self.b.copy()
        c_B = c[basic_columns]
        zeroth_row = -1 * np.hstack([np.dot(c_B, self.A), np.zeros(self.A.shape[0])])
        zeroth_col = b
        zeroth_element = -1 * np.sum(b)
        
        rest = A.copy()
        
        ph1_tableau = np.block([
            [zeroth_element, zeroth_row],
            [zeroth_col.reshape(1, zeroth_col.shape[0]).T, rest]
        ])
        
        iters = 0
        
        while (ph1_tableau[0, 1:] < 0).any():

            j = np.where(ph1_tableau[0, 1:] < 0)[0][0]     # incoming basis direction
            theta = [i for i in range(1, ph1_tableau.shape[0]) if ph1_tableau[i, j + 1] > 0][0]
            for i in range(1, ph1_tableau.shape[0]): 
                if ph1_tableau[i, j + 1] > 0 and ph1_tableau[i, 0] / ph1_tableau[i, j + 1] >= 0:
                    if ph1_tableau[i, 0] / ph1_tableau[i, j + 1]  < ph1_tableau[theta, 0] / ph1_tableau[theta, j + 1]:
                        theta = i


            basic_columns[theta - 1] = j
            pivot_row = theta          # index of direction which will exit the basis matrix
            pivot_col = j + 1          # direction which will enter the basis

            ph1_tableau[pivot_row, :] = ph1_tableau[pivot_row, :] / ph1_tableau[pivot_row, pivot_col]

            for i in range(ph1_tableau.shape[0]):
                if i == pivot_row:
                    continue        
                ph1_tableau[i, :] = ph1_tableau[i, :] - (ph1_tableau[i, pivot_col] / ph1_tableau[pivot_row, pivot_col]) * ph1_tableau[pivot_row, :]


            iters+=1
            if iters == self.max_iter:
                raise RuntimeError("Cycling encountered! Method could not converge in max_iter = %d iterations. Terminating..." %(self.max_iter))

        if ph1_tableau[0, 0] > 0:
            raise RuntimeError("Given LP is infeasible!")
            
        elif ph1_tableau[0, 0] == 0:
            
            if (basic_columns < self.A.shape[1]).all():

                return ph1_tableau[1:, :self.A.shape[1]+1], basic_columns
            
            else:
                
                while True:
                    av_inbasis_at = np.where(basic_columns >= self.A.shape[1])[0].tolist()
                    
                    
                    if (ph1_tableau[av_inbasis_at[0] + 1, 1:self.A.shape[1] + 1] == 0).all():
                        ph1_tableau = np.delete(ph1_tableau, (av_inbasis_at[0] + 1), axis=0)
                        self.A = np.delete(self.A, av_inbasis_at[0], axis=0)
                        self.b = np.delete(self.b, av_inbasis_at[0])
                        basic_columns = np.delete(basic_columns, av_inbasis_at[0])

                    else:
                        pivot_row = av_inbasis_at[0] + 1
                        pivot_col = np.where(ph1_tableau[pivot_row, 1:] != 0) + 1
                        ph1_tableau[pivot_row, :] = ph1_tableau[pivot_row, :] / ph1_tableau[pivot_row, pivot_col]
                        for i in range(ph1_tableau.shape[0]):
                            if i == pivot_row:
                                continue        
                            ph1_tableau[i, :] = ph1_tableau[i, :] - (ph1_tableau[i, pivot_col] / ph1_tableau[pivot_row, pivot_col]) * ph1_tableau[pivot_row, :]
                        basic_columns[av_inbasis_at[0]] = pivot_col - 1
                    av_inbasis_at = np.where(basic_columns >= self.A.shape[1])[0].tolist()
                    if len(av_inbasis_at) == 0:
                        break

        return ph1_tableau[1:, :(self.A.shape[1] + 1)], basic_columns
    
    def run_phase2(self, tableau, basic_columns):
        self.tableau = tableau.copy()
        iters = 0
        while (tableau[0, 1:] < 0).any():
            j = np.where(tableau[0, 1:] < 0)[0][0]     # incoming basis direction
            theta = [i for i in range(1, tableau.shape[0]) if tableau[i, j + 1] > 0][0]
            for i in range(1, tableau.shape[0]): 
                if tableau[i, j + 1] > 0 and tableau[i, 0] / tableau[i, j + 1] >= 0:
                    if tableau[i, 0] / tableau[i, j + 1]  < tableau[theta, 0] / tableau[theta, j + 1]:
                        theta = i


            basic_columns[theta - 1] = j

            pivot_row = theta          # index of direction which will exit the basis matrix
            pivot_col = j + 1          # direction which will enter the basis

            tableau[pivot_row, :] = tableau[pivot_row, :] / tableau[pivot_row, pivot_col]

            for i in range(tableau.shape[0]):
                if i == pivot_row:
                    continue        
                tableau[i, :] = tableau[i, :] - (tableau[i, pivot_col] / tableau[pivot_row, pivot_col]) * tableau[pivot_row, :]

            iters+=1
            if iters == self.max_iter:
                raise RuntimeError("Method could not converge in max_iter = %d iterations. Terminating method...!\n\n" %(self.max_iter))
                

        self.solution = LPSolution(self.num_vars, {basic_columns[i]: tableau[1:, 0][i] for i in range(len(basic_columns))}, -1 * tableau[0, 0])

        return self.solution
        

    def run_simplex2(self):
        lower_half_tableau, initial_basis = self.run_phase1()
        b = self.b.copy()
        B = self.get_first_B(initial_basis)
        c_B = self.c[initial_basis]
        zeroth_element = -1 * np.dot(c_B, np.linalg.solve(B, b))
        zeroth_row = self.c - np.dot(c_B, np.dot(np.linalg.inv(B), self.A))

        tableau = np.vstack((np.hstack((zeroth_element, zeroth_row)), lower_half_tableau))

        
        self.solution = self.run_phase2(tableau, initial_basis)
        
        return self.solution

In [21]:
constraints3, obj3 = nf2PrimalLP("nf1.dat")
splex = Simplex(14, 20, obj3, constraints3, 1000)
sol = splex.run_simplex2()
print(str(sol))

	Solution to the LP is as follows:

		optim	:=	-26.0

		x_1*	:=	16.0
		x_2*	:=	10.0
		x_3*	:=	8.0
		x_4*	:=	12.0
		x_5*	:=	4.0
		x_6*	:=	14.0
		x_7*	:=	0.0
		x_8*	:=	19.0
		x_9*	:=	7.0
		x_10*	:=	7.0
		x_11*	:=	0.0
		x_12*	:=	3.0
		x_13*	:=	2.0
		x_14*	:=	0.0
		x_15*	:=	0.0
		x_16*	:=	0.0
		x_17*	:=	9.0
		x_18*	:=	1.0
		x_19*	:=	0.0
		x_20*	:=	0.0



In [22]:
constraints4, obj4 = nf2PrimalLP("nf2.dat")
splex = Simplex(39, 58, obj4, constraints4, 1000)
sol = splex.run_simplex2()
print(str(sol))

	Solution to the LP is as follows:

		optim	:=	-29.0

		x_1*	:=	11.0
		x_2*	:=	8.0
		x_3*	:=	10.0
		x_4*	:=	10.0
		x_5*	:=	4.0
		x_6*	:=	3.0
		x_7*	:=	5.0
		x_8*	:=	6.0
		x_9*	:=	3.0
		x_10*	:=	6.0
		x_11*	:=	0.0
		x_12*	:=	12.0
		x_13*	:=	1.0
		x_14*	:=	3.0
		x_15*	:=	7.0
		x_16*	:=	0.0
		x_17*	:=	4.0
		x_18*	:=	0.0
		x_19*	:=	4.0
		x_20*	:=	9.0
		x_21*	:=	3.0
		x_22*	:=	3.0
		x_23*	:=	4.0
		x_24*	:=	5.0
		x_25*	:=	4.0
		x_26*	:=	7.0
		x_27*	:=	9.0
		x_28*	:=	2.0
		x_29*	:=	15.0
		x_30*	:=	0.0
		x_31*	:=	7.0
		x_32*	:=	0.0
		x_33*	:=	8.0
		x_34*	:=	0.0
		x_35*	:=	0.0
		x_36*	:=	0.0
		x_37*	:=	0.0
		x_38*	:=	0.0
		x_39*	:=	5.0
		x_40*	:=	4.0
		x_41*	:=	5.0
		x_42*	:=	5.0
		x_43*	:=	0.0
		x_44*	:=	6.0
		x_45*	:=	12.0
		x_46*	:=	0.0
		x_47*	:=	21.0
		x_48*	:=	0.0
		x_49*	:=	0.0
		x_50*	:=	1.0
		x_51*	:=	0.0
		x_52*	:=	0.0
		x_53*	:=	0.0
		x_54*	:=	0.0
		x_55*	:=	0.0
		x_56*	:=	0.0
		x_57*	:=	0.0
		x_58*	:=	0.0



In [23]:
constraints, obj = nf2DualLP("nf1.dat")

In [24]:
splex = Simplex(10, 20, obj, constraints, 1000)
sol = splex.run_simplex2()
print(str(sol))

	Solution to the LP is as follows:

		optim	:=	26.0

		x_1*	:=	0.0
		x_2*	:=	0.0
		x_3*	:=	0.0
		x_4*	:=	1.0
		x_5*	:=	0.0
		x_6*	:=	1.0
		x_7*	:=	0.0
		x_8*	:=	0.0
		x_9*	:=	0.0
		x_10*	:=	0.0
		x_11*	:=	1.0
		x_12*	:=	1.0
		x_13*	:=	0.0
		x_14*	:=	0.0
		x_15*	:=	0.0
		x_16*	:=	0.0
		x_17*	:=	0.0
		x_18*	:=	0.0
		x_19*	:=	0.0
		x_20*	:=	0.0



In [25]:
constraints2, obj2 = nf2DualLP("nf2.dat")
splex = Simplex(29, 68, obj2, constraints2, 1000)
sol = splex.run_simplex2()
print(str(sol))

	Solution to the LP is as follows:

		optim	:=	29.0

		x_1*	:=	1.0
		x_2*	:=	0.0
		x_3*	:=	1.0
		x_4*	:=	0.0
		x_5*	:=	0.0
		x_6*	:=	1.0
		x_7*	:=	1.0
		x_8*	:=	0.0
		x_9*	:=	0.0
		x_10*	:=	0.0
		x_11*	:=	0.0
		x_12*	:=	0.0
		x_13*	:=	0.0
		x_14*	:=	0.0
		x_15*	:=	0.0
		x_16*	:=	0.0
		x_17*	:=	0.0
		x_18*	:=	0.0
		x_19*	:=	0.0
		x_20*	:=	0.0
		x_21*	:=	0.0
		x_22*	:=	0.0
		x_23*	:=	0.0
		x_24*	:=	0.0
		x_25*	:=	0.0
		x_26*	:=	0.0
		x_27*	:=	0.0
		x_28*	:=	0.0
		x_29*	:=	0.0
		x_30*	:=	0.0
		x_31*	:=	1.0
		x_32*	:=	0.0
		x_33*	:=	0.0
		x_34*	:=	0.0
		x_35*	:=	0.0
		x_36*	:=	0.0
		x_37*	:=	0.0
		x_38*	:=	0.0
		x_39*	:=	0.0
		x_40*	:=	0.0
		x_41*	:=	0.0
		x_42*	:=	0.0
		x_43*	:=	0.0
		x_44*	:=	0.0
		x_45*	:=	0.0
		x_46*	:=	0.0
		x_47*	:=	0.0
		x_48*	:=	0.0
		x_49*	:=	0.0
		x_50*	:=	0.0
		x_51*	:=	0.0
		x_52*	:=	0.0
		x_53*	:=	0.0
		x_54*	:=	0.0
		x_55*	:=	0.0
		x_56*	:=	0.0
		x_57*	:=	0.0
		x_58*	:=	0.0
		x_59*	:=	0.0
		x_60*	:=	0.0
		x_61*	:=	0.0
		x_62*	:=	0.0
		x_63*	:=	0.0
		x_64*	:=	